# Bibliotecas

In [1]:
import numpy as np

# Abrindo dados

In [2]:
x = np.float64(np.load(open('data/cnn/x.npy', 'rb')))
y = np.int8(np.load(open('data/cnn/y.npy', 'rb')))

In [3]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

2024-04-04 21:38:27.419901: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 21:38:27.451415: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 21:38:27.451467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 21:38:27.452459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 21:38:27.458791: I tensorflow/core/platform/cpu_feature_guar

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

# Criar Modelo

In [6]:
from keras import layers, models

input = layers.Input(shape=(501, 6, 1))

timeData = layers.Cropping2D(((0, 500), (2, 0)))(input)
timeData = layers.Flatten()(timeData)

m5Data = layers.Cropping2D(((1, 200), (0, 0)))(input)
m5Data = layers.Conv2D(50, (24, 5))(m5Data)
m5Data = layers.MaxPooling2D((2, 2))(m5Data)
m5Data = layers.Reshape((138, 50))(m5Data)
m5Data = layers.Conv1D(100, 24)(m5Data)
m5Data = layers.MaxPooling1D(2)(m5Data)
m5Data = layers.Conv1D(200, 8)(m5Data)
m5Data = layers.MaxPooling1D(2)(m5Data)
m5Data = layers.Flatten()(m5Data)

d1Data = layers.Cropping2D(((301, 0), (0, 0)))(input)
d1Data = layers.Conv2D(50, (24, 5))(d1Data)
d1Data = layers.MaxPooling2D((2, 2))(d1Data)
d1Data = layers.Reshape((88, 50))(d1Data)
d1Data = layers.Conv1D(100, 24)(d1Data)
d1Data = layers.MaxPooling1D(2)(d1Data)
d1Data = layers.Conv1D(200, 8)(d1Data)
d1Data = layers.MaxPooling1D(2)(d1Data)
d1Data = layers.Flatten()(d1Data)

concat = layers.Concatenate()([timeData, m5Data, d1Data])

modelo = layers.Dropout(0.5)(concat)
modelo = layers.Dense(5000, 'relu')(modelo)
modelo = layers.Dropout(0.5)(modelo)
modelo = layers.Dense(5000, 'relu')(modelo)
modelo = layers.Dense(3, 'softmax')(modelo)

modelo = models.Model(input, modelo)
modelo.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
modelo.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 501, 6, 1)]          0         []                            
                                                                                                  
 cropping2d_4 (Cropping2D)   (None, 300, 6, 1)            0         ['input_2[0][0]']             
                                                                                                  
 cropping2d_5 (Cropping2D)   (None, 200, 6, 1)            0         ['input_2[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 277, 2, 50)           6050      ['cropping2d_4[0][0]']        
                                                                                            

# Treinar Modelo

In [7]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/customModel/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=15, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('/mnt/c/Users/aurel/OneDrive/Documentos/Projetos/tradingPredictor/models/CustomModel2.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=500,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)

Epoch 1/500
234/234 [==============================] - 18s 73ms/step - loss: 1.2727 - accuracy: 0.4051 - val_loss: 1.0384 - val_accuracy: 0.4471
Epoch 2/500
234/234 [==============================] - 14s 61ms/step - loss: 1.0263 - accuracy: 0.4462 - val_loss: 1.0251 - val_accuracy: 0.4503
Epoch 3/500
234/234 [==============================] - 15s 64ms/step - loss: 0.9998 - accuracy: 0.4732 - val_loss: 0.9803 - val_accuracy: 0.4964
Epoch 4/500
234/234 [==============================] - 15s 62ms/step - loss: 0.9640 - accuracy: 0.5097 - val_loss: 0.9070 - val_accuracy: 0.5523
Epoch 5/500
234/234 [==============================] - 15s 63ms/step - loss: 0.8852 - accuracy: 0.5745 - val_loss: 0.7960 - val_accuracy: 0.6357
Epoch 6/500
234/234 [==============================] - 15s 62ms/step - loss: 0.7509 - accuracy: 0.6666 - val_loss: 0.6266 - val_accuracy: 0.7541
Epoch 7/500
234/234 [==============================] - 14s 61ms/step - loss: 0.6178 - accuracy: 0.7449 - val_loss: 0.4912 - val_ac